In [31]:
# %load ../imports.py
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt

In [32]:
DATASET_PATH = '../../dataset/private/housing.csv'
COLUMN_NAMES = [
    'CR', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
    'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV'
]
FEATURES = [
    'CR', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
    'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT',
]
TARGET = 'MEDV'

In [33]:
data = pd.read_csv(
    DATASET_PATH,
    header=None, 
    names=COLUMN_NAMES, 
    delim_whitespace=True
)
print("Null values:", data.isnull().any(), sep='\n')
data.head()

Null values:
CR         False
ZN         False
INDUS      False
CHAS       False
NOX        False
RM         False
AGE        False
DIS        False
RAD        False
TAX        False
PTRATIO    False
B          False
LSTAT      False
MEDV       False
dtype: bool


,CR,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [34]:
import sklearn.metrics as sklm
def scoring_procedure(y_true, y_pred, header=''):
    abs_err = abs(y_true - y_pred)
    mse = sklm.mean_squared_error(y_true, y_pred)
    report = f"""
    {header}
    MSE: {mse:.3f}
    RMSE: {np.sqrt(mse):.3f} 
    R2: {sklm.r2_score(y_true, y_pred):.3f}
    MAPE: {sklm.mean_absolute_percentage_error(y_true, y_pred):.3f}
    """
    print(report)

In [35]:
X = data[FEATURES]
y = data[TARGET]

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

lr = LinearRegression()
lr.fit(X_train, y_train)

# За кадром был произведён гридсёрч от 10^-4 до 10^4 по 20 точек и были получены оценки гиперпараметров
ridge = Ridge(alpha=1.5)
ridge.fit(X_train, y_train)

lasso = Lasso(alpha=0.2)
lasso.fit(X_train, y_train)

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.1)
elastic_net.fit(X_train, y_train)

ElasticNet(alpha=0.1, l1_ratio=0.1)

In [39]:
scoring_procedure(y_test, lr.predict(X_test), 'Linear')
scoring_procedure(y_test, ridge.predict(X_test), 'Ridge')
scoring_procedure(y_test, lasso.predict(X_test), 'Lasso')
scoring_procedure(y_test, elastic_net.predict(X_test), 'ElasticNet')


    Linear
    MSE: 21.517
    RMSE: 4.639 
    R2: 0.711
    MAPE: 0.165
    

    Ridge
    MSE: 21.564
    RMSE: 4.644 
    R2: 0.711
    MAPE: 0.165
    

    Lasso
    MSE: 23.797
    RMSE: 4.878 
    R2: 0.681
    MAPE: 0.168
    

    ElasticNet
    MSE: 22.373
    RMSE: 4.730 
    R2: 0.700
    MAPE: 0.162
    


In [48]:
coef_table = DataFrame({
    'Feature': FEATURES,
    'Linear': lr.coef_,
    'Ridge': ridge.coef_,
    'Lasso': lasso.coef_,
    "ElasticNet": elastic_net.coef_
})

display(
    coef_table.style.background_gradient(cmap='coolwarm')
)

,Feature,Linear,Ridge,Lasso,ElasticNet
0,CR,-1.146914,-1.127306,-0.545099,-0.889413
1,ZN,0.834326,0.802341,0.126932,0.473484
2,INDUS,0.339407,0.297022,-0.000000,-0.072494
3,CHAS,0.791636,0.799909,0.764918,0.872350
4,NOX,-1.784727,-1.732504,-0.727743,-1.085824
5,RM,2.847839,2.857541,2.983582,2.908521
6,AGE,-0.304293,-0.308619,-0.000000,-0.310787
7,DIS,-2.915625,-2.856674,-1.406049,-2.013988
8,RAD,2.111400,1.988528,0.000000,0.830099
9,TAX,-1.465200,-1.357243,-0.000000,-0.519067


### Анализ признаков

- Наиболее важные признаки по абсолютному значению коэффициентов:

    - LSTAT, RM, DIS, PTRATIO, NOX — оказывают наибольшее влияние во всех моделях.

- Наименее важные признаки:

    - INDUS, AGE, ZN, CHAS — имеют малое или зануленное влияние, особенно в Lasso.

- Сравнение с ручным отбором

    - Вручную были выбраны RM, PTRATIO, TAX, NOX, CR, AGE, ZN

- Мультиколлинеарность:

    - Указание на мультиколлинеарность — обнуление коэффициентов в Lasso (например, INDUS, AGE, RAD, TAX). Это значит, что Lasso исключает коррелирующие признаки.

### Анализ моделей

- Linear и Ridge сохраняют почти все признаки, лишь слегка сглаживая веса.

- Lasso активно зануляет признаки, подходит для отбора важных переменных.

- ElasticNet комбинирует подходы Lasso и Ridge: частично зануляет и частично сглаживает, оставляя более интерпретируемую модель с меньшим переобучением.

# Итог

- Lasso выявил ключевые признаки и устранил малозначимые, что может говорить о наличии мультиколлинеарности.

- ElasticNet балансирует между селекцией и регуляризацией.

- LSTAT и RM — наиболее стабильные и значимые предикторы во всех моделях.